In [ ]:
from Nozzle_1D import Nozzle
import matplotlib.pyplot as plt
import numpy as np
import pickle

In [ ]:
tolerance = 1e-12

In [ ]:
np.where(np.linspace(0,1,1000)>1)

In [ ]:
# Initialize MEDIUM ISENTROPIC CASE
MEDIUM_ISENTROPIC = Nozzle("inputs/Medium_inputs.nml")
MEDIUM_ISENTROPIC.CFL = .1
MEDIUM_ISENTROPIC.p_back = 120
MEDIUM_ISENTROPIC.local_timestep = True
MEDIUM_ISENTROPIC.iter_max = 100000
MEDIUM_ISENTROPIC.flat_initial_mach = True
# RUN MEDIUM SIMULATION
p_compute_Medium,u_compute_Medium,rho_compute_Medium,p_exact_Medium,\
    u_exact_Medium,rho_exact_Medium,convergence_history_Medium = MEDIUM_ISENTROPIC.RUN_SIMULATION(verbose = True)

In [ ]:
plt.plot(MEDIUM_ISENTROPIC.temp_plot)

In [ ]:
plt.semilogy(convergence_history_Medium)

In [ ]:
p_compute_Medium[-1]

In [ ]:
#plt.plot(p_compute_Medium,"-")
plt.plot(MEDIUM_ISENTROPIC.x[1:],p_compute_Medium[1:-1],"-")
plt.title("Pressure plot--1D Normal Shock")
plt.xlabel(r"x")
plt.ylabel("Pressure (kPa)")

In [ ]:
# Initialize MEDIUM ISENTROPIC CASE
MEDIUM_ISENTROPIC = Nozzle("inputs/Medium_inputs.nml")
MEDIUM_ISENTROPIC.CFL = .03
# RUN MEDIUM SIMULATION
p_compute_Medium,u_compute_Medium,rho_compute_Medium,p_exact_Medium,\
    u_exact_Medium,rho_exact_Medium,convergence_history_Medium = MEDIUM_ISENTROPIC.RUN_SIMULATION(verbose = True,convergence_criteria=tolerance,jameson_damping = False,iter_max=100000,first_order=False)

In [ ]:
temp = (MEDIUM_ISENTROPIC.NI-1)//2
plt.plot(MEDIUM_ISENTROPIC.x[temp-2:temp+3],rho_compute_Medium[temp-2:temp+3],"-o")
plt.title("Pressure plot--1D Isentropic nozzle")
plt.xlabel(r"x")
plt.ylabel("Pressure (kPa)")

In [ ]:
plt.plot(MEDIUM_ISENTROPIC.x[1:],np.sqrt(p_compute_Medium[1:-1]/rho_compute_Medium[1:-1]))
plt.title("Pressure plot--1D Isentropic nozzle")
plt.xlabel(r"x")
plt.ylabel("Pressure (kPa)")

In [ ]:
convergence_history_Medium

In [ ]:
plt.semilogy(convergence_history_Medium)

# Add labels and title
plt.xlabel('Iterations (100x)')
plt.ylabel('Error (log scale)')
plt.title('Convergence History (Isentropic)')

# Add legend
plt.legend(["Density", "Velocity", "Pressure"])

# Add grid for better readability
plt.grid(True, which="both", linestyle="--", linewidth=0.7)

In [ ]:
# Initialize MEDIUM NON-ISENTROPIC CASE
MEDIUM_NON_ISENTROPIC = Nozzle("inputs/Medium_inputs.nml")

# Manually change back pressure
MEDIUM_NON_ISENTROPIC.p_back = 120 #kPa

# RUN MEDIUM SIMULATION
p_compute_Medium,u_compute_Medium,rho_compute_Medium,p_exact_Medium,\
    u_exact_Medium,rho_exact_Medium,convergence_history_Medium = MEDIUM_NON_ISENTROPIC.RUN_SIMULATION(verbose = True,convergence_criteria=tolerance)

In [ ]:
plt.plot(MEDIUM_NON_ISENTROPIC.x[1:],p_compute_Medium[1:-1])
plt.title("Pressure plot--1D nozzle with shocks")
plt.xlabel(r"x")
plt.ylabel("Pressure (kPa)")

In [ ]:
plt.semilogy(convergence_history_Medium)

# Add labels and title
plt.xlabel('Iterations (100x)')
plt.ylabel('Error (log scale)')
plt.title('Convergence History (Non-Isentropic)')

# Add legend
plt.legend(["Density", "Velocity", "Pressure"])

# Add grid for better readability
plt.grid(True, which="both", linestyle="--", linewidth=0.7)

In [ ]:
import numpy as np
import scipy.sparse as sp
import scipy.sparse.linalg as spla

def poisson_2d(n):
    h = 1 / (n + 1)  # Grid spacing
    N = n * n  # Total unknowns

    # Discrete Laplacian operator (5-point stencil)
    main_diag = 4 * np.ones(N)
    off_diag = -1 * np.ones(N - 1)
    off_diag[np.arange(1, N) % n == 0] = 0  # Fix for boundaries
    far_off_diag = -1 * np.ones(N - n)

    A = sp.diags([main_diag, off_diag, off_diag, far_off_diag, far_off_diag], 
                 [0, -1, 1, -n, n], shape=(N, N), format="csr")

    # Right-hand side (forcing term, f)
    x = np.linspace(h, 1 - h, n)
    y = np.linspace(h, 1 - h, n)
    X, Y = np.meshgrid(x, y, indexing="ij")
    F = np.sin(np.pi * X) * np.sin(np.pi * Y)  # Example forcing function
    b = (h ** 2) * F.ravel()  # Flatten to 1D

    # Solve the linear system
    u = spla.spsolve(A, b)

    # Reshape solution back to (n, n)
    return u.reshape((n, n))

# Example: Solve for a 10x10 grid
solution = poisson_2d(100)
plt.imshow(solution)